Links
* [Keras datasets](https://keras.io/datasets/)
* 

1. Discuss dataset
2. Explain embedding
3. Define fully connected model
4. Define convolution model

In [2]:
import numpy as np
import keras
from keras.datasets import imdb
from matplotlib import pyplot as plt
%matplotlib inline

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
# load the dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data()
print('There are {:,} training and {:,} test examples'.format(
    X_train.shape[0], X_test.shape[0]))

There are 25,000 training and 25,000 test examples


In [7]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

Get ratings for both training and test data

In [8]:
unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{0: 25000, 1: 25000}

Get word to integer dictionary and create the inverse mapping

In [9]:
imdb_word_int = imdb.get_word_index()
imdb_int_word = {int_val:word for word, int_val in imdb_word_int.items()}

Display the first review using only words. The punctuation has been removed

In [10]:
review_idx = 0
review_int = X_train[review_idx]
review_rating = y_train[review_idx]
review = ' '.join([imdb_int_word[int_val] for int_val in review_int])
print('rating {}'.format(review_rating))
review

rating 1


"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room titillate it so heart shows to years of every never going villaronga help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of gilmore's br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but wh

Display the unique number of words in the data set

In [11]:
len(np.unique(np.hstack(X)))

88585